In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
batting=pd.read_csv('/kaggle/input/career-hitting/batting.csv').drop(columns='Unnamed: 0')
batting

In [ ]:
master=pd.read_excel('/kaggle/input/correct/correct.xlsx')
master=master[['IDPLAYER','PLAYERNAME']]
master

In [ ]:
master['IDPLAYER'].str.contains('abreubo01').any()

In [ ]:
players=[]
for player in batting['player_id']:
    if master['IDPLAYER'].str.contains(player).any():
        players.append(player)

In [ ]:
len(players)

In [ ]:
len(batting)

Dataset of player names is inadequate because it only accounts for 410 of 3803 player IDs we need.

In [ ]:
hof=pd.read_csv('/kaggle/input/baseball-databank/HallOfFame.csv')
hof

HOF dataset has a little more than the batting dataset- this is probably due to pitchers or managers.

In [ ]:
#df of all players included in in HOF dataset
ballot=batting[batting['player_id'].isin(hof['playerID'].tolist())]
ballot

In [ ]:
#df of all players not in HOF dataset
absent=batting[-batting['player_id'].isin(hof['playerID'].tolist())]
absent

In [ ]:
absent.sort_values(by='rbi',ascending=False)

Lot of these players are current players (dataset is updated as of 2016), and many would never get votes for the hall. For example, let's see how many of these players have at least 2000 hits.

In [ ]:
hits=absent[absent['h']>=2000]
hits

In [ ]:
len(hits)

Only 44 have 2000 hits, and I recognize pretty much all of the players because they're either still playing or were not yet eligible for the hall of fame as of 2016 (have to be retired for at least five years).

In [ ]:
hof[hof['playerID'].isin(batting['player_id'].tolist())]

Problem with appending the HOF dataset is that players will show up more than once for each time they show up on the ballot. It took Craig Biggio years to finally reach the thershold of votes to be inducted, but Hank Aaron obviously was inducted the first year he was eligible.

Going to have to manipulate the hof dataset such that we can append it to each player's hitting totals.

In [ ]:
hof.columns

In [ ]:
hof['category'].value_counts()

In [ ]:
hof['needed_note'].value_counts()

In [ ]:
hof=hof[hof['category'].isin(['Player'])]
hof=hof.drop(columns=['needed_note'])
hof

In [ ]:
hof['percent']=hof['votes']/hof['ballots']
hof['threshold']=hof['needed']/hof['ballots']
hof[hof['inducted'].isin(['Y'])]

In [ ]:
type(hof['inducted'][0])

In [ ]:
if hof['inducted'].str.contains('Y').any():
    print('yes')

To get voting data into one column, we're going to show the average % of ballot a player received through the years, and the average % threshold needed.

In [ ]:
df=hof[hof['playerID'].isin(['youngro01'])]
if df['inducted'].str.contains('Y').any():
    inducted=df[df['inducted'].isin(['Y'])].reset_index()
    print(inducted['percent'][0])
#     threshold=inducted['threshold'][0]

In [ ]:
#how to check if there are any null values in column
#df['your column name'].isnull().values.any()

In [ ]:
hof_clean=pd.DataFrame()
for player in hof['playerID'].value_counts().index:
    playerdf=pd.DataFrame()
    
    #extract player data
    df=hof[hof['playerID'].isin([player])].reset_index()
        
    #store information to append
#     induction=df[df['inducted'].isin(['Y'])].reset_index()
    year=df.max()['yearid']
    #voter=df['votedBy'][0]
    
#     df['percent']=df['votes']/df['ballots']
#     df['threshold']=df['needed']/df['ballots']
    
#     ballots=df.sum()['ballots']/len(df)
#     needed=df.sum()['needed']/len(df)
#     votes=df.sum()['votes']/len(df)
    attempts=len(df)
    if df['inducted'].str.contains('Y').any():
        inducted=df[df['inducted'].isin(['Y'])].reset_index()
        percent=inducted['percent'][0]
        threshold=inducted['threshold'][0]
        votedBy=inducted['votedBy'][0]
        playerdf=playerdf.append({'year':year,'player_id':player,'percent':percent,'threshold':
                                threshold,'years':attempts,'votedBy':votedBy,'inducted':'Y'},ignore_index=True)
    else:
#         year=df.max()['yearid']
#         voter=df['votedBy'].value_counts().index[0]
#         ballots=df.sum()['ballots']/len(df)
#         needed=df.sum()['needed']/len(df)
#         votes=df.sum()['votes']/len(df)
#         attempts=len(df)
        max_percent=df.max()['percent']
        threshold=df['threshold'][df['percent'].idxmax()]
        votedBy=df['votedBy'].value_counts().index[0]
        playerdf=playerdf.append({'year':year,'player_id':player,'percent':max_percent,'threshold':
                                threshold,'years':attempts,'votedBy':votedBy,'inducted':'N'},ignore_index=True)
    hof_clean=hof_clean.append(playerdf)

In [ ]:
hof_clean=hof_clean.reset_index().drop(columns='index')
hof_clean

In [ ]:
hof_clean[-hof_clean['votedBy'].isin(['BBWAA'])]['threshold'].isnull().value_counts()

Of players not voted in by BBWAA, 135/165 did no go through standard voting procedures, or the data was at least not recorded. Let's leave these other voting routes in the dataset, but keep in mind that we may want to take them out.

In [ ]:
combined=batting.join(hof_clean.set_index('player_id'), on='player_id')
combined

In [ ]:
#take out players that aren't in hof dataset
clean=combined[-combined['inducted'].isnull()]
clean

In [ ]:
clean['inducted'].value_counts()

235 players in mlb hof, so this is a pretty good dataset.

In [ ]:
clean[clean['votedBy'].isin(['BBWAA'])&clean['inducted'].isin(['Y'])].sort_values(by='h')

We still have a lot of pitchers in the dataset- whose haitting statistics shouldn't influence the model.

In [ ]:
pitchers=pd.read_csv('/kaggle/input/baseball-databank/Pitching.csv')
pitchers

A lot of players in the old days, and even today, will pitch even if they're a position player. Thus, they show up on the pithers list even though they're hall of famers because of their hitting. Account for this by cutting all with less than 100 hits.

In [ ]:
pitchers_list=[]
for player in clean['player_id']:
    if pitchers['playerID'].str.contains(player).any():
        df=clean[clean['player_id'].isin([player])].reset_index()
        
        #weed out hitters that still show up, if they have less than 1000 hits then they are not in the
        #hof because of their hitting statistics
        if df['h'][0] <1000:
            pitchers_list.append(player)

In [ ]:
pitchers_list

In [ ]:
hitters=clean[-clean['player_id'].isin(pitchers_list)]
hitters

In [ ]:
hitters['inducted'].value_counts()

67 HOFers taken out, this seems accurate in terms of getting rid of pitchers.

In [ ]:
hitters.sort_values(by='hr',ascending=False).head(10)

Top 10 home run hitters is correct- just making sure I'm not missing a significant amount of data. Notice that players like Alex Rodriguez and Albert Pujols are not on here- this is because as of 2016 they were still active players or were not yet eligible for the hall of fame.

In [ ]:
#reset indices
hitters=hitters.reset_index().drop(columns='index')

In [ ]:
hitters.to_csv('hitters2.csv')